In [1]:
import copy
from functools import reduce

import numpy as np

from utils.get_input import get_input

# inp = Path("./test_input").read_text()
inp = get_input(11)

In [2]:
# Parse
monkeys = inp.split("\n\n")
monkeys_parsed = [{
    "items": list(map(int, ((l := m.splitlines())[1].split(":")[1].split(",")))),
    "op": ((x := l[2].split("= old")[1].strip().split(" "))[0], int(x[1]) if x[1].isdigit() else x[1]),
    "test": int(l[3].split("divisible by ")[1]),
    True: int(l[4].split("monkey ")[1]),
    False: int(l[5].split("monkey ")[1])
} for m in monkeys]

In [3]:
# Part 1
monkeys1 = copy.deepcopy(monkeys_parsed)
inspected = [0] * len(monkeys1)


def play_round(ms):
    for i, m in enumerate(ms):
        for item in m["items"]:
            inspected[i] += 1
            # Do worry op
            match m["op"]:
                case ("*", int(v)):
                    new_worry = item * v
                case ("+", int(v)):
                    new_worry = item + v
                case ("*", str(v)):
                    new_worry = item * item
                case ("+", str(v)):
                    new_worry = item + item
            # Bored
            new_worry //= 3
            # Choose target
            target = m[new_worry % m["test"] == 0]
            # Throw
            ms[target]["items"].append(new_worry)
        m["items"] = []


for i in range(20):
    play_round(monkeys1)

res = np.prod(sorted(inspected)[-2:])
print(res)

110888


In [4]:
# Part 2
monkeys1 = copy.deepcopy(monkeys_parsed)
inspected = [0] * len(monkeys1)

tests = [m["test"] for m in monkeys1]


def chinese_remainder(m, a):
    sum = 0
    prod = reduce(lambda acc, b: acc * b, m)
    for n_i, a_i in zip(m, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1: return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0: x1 += b0
    return x1


def play_round(ms):
    for i, m in enumerate(ms):
        for item in m["items"]:
            inspected[i] += 1
            # Do worry op
            match m["op"]:
                case ("*", int(v)):
                    new_worry = item * v
                case ("+", int(v)):
                    new_worry = item + v
                case ("*", str(v)):
                    new_worry = item * item
                case ("+", str(v)):
                    new_worry = item + item

            worries = [new_worry % t for t in tests]
            new_worry = chinese_remainder(tests, worries)
            # Choose target
            target = m[new_worry % m["test"] == 0]
            # Throw
            ms[target]["items"].append(new_worry)
        m["items"] = []


for i in range(10000):
    play_round(monkeys1)

res = sorted(inspected)[-2:]
print(res[0] * res[1])

25590400731
